# GenAI Incident Report Prototype

Prototypes the LLM-based incident report generation before porting to `cv_backend/routers/report.py`.

## 1. Setup

In [ ]:
!pip install openai pydantic python-dotenv --quiet
import os
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import uuid, time, json

## 2. Data Schemas

In [ ]:
class SensorSnapshot(BaseModel):
    heart_rate: Optional[float] = None
    spo2: Optional[float] = None
    step_count: Optional[int] = None
    skin_temperature: Optional[float] = None
    timestamp: int

class AnomalyEvent(BaseModel):
    type: str
    confidence: float
    track_id: int
    timestamp: int
    duration_seconds: Optional[float] = None

class IncidentReport(BaseModel):
    report_id: str
    generated_at: int
    severity: str
    summary: str
    vitals_assessment: str
    cv_assessment: str
    recommended_action: str

print('Schemas defined.')

## 3. Sample Data

In [ ]:
# TODO: fine-tune — swap in real event data once CV pipeline is running
sample_anomaly = AnomalyEvent(
    type='FALL',
    confidence=0.87,
    track_id=3,
    timestamp=int(time.time() * 1000),
    duration_seconds=2.5,
)

sample_sensor = SensorSnapshot(
    heart_rate=105.0,
    spo2=96.0,
    step_count=800,
    skin_temperature=37.1,
    timestamp=int(time.time() * 1000),
)

location_context = 'gym'
print('Sample data ready.')

## 4. Prompt Engineering

In [ ]:
# TODO: fine-tune — expand system prompt with domain knowledge, severity criteria
SYSTEM_PROMPT = (
    'You are a medical triage assistant analyzing wearable sensor data and '
    'computer-vision anomaly events. Respond with ONLY these fields, one per line, '
    'in the format "field: value":\n'
    'severity (LOW/MODERATE/HIGH/CRITICAL)\n'
    'summary (2-3 sentences)\n'
    'vitals_assessment\n'
    'cv_assessment\n'
    'recommended_action'
)

def build_user_prompt(anomaly: AnomalyEvent, sensor: SensorSnapshot, location: str) -> str:
    # TODO: fine-tune — add historical baseline comparison, user age/conditions
    return (
        f'Anomaly detected: {anomaly.type}.\n'
        f'Confidence: {anomaly.confidence:.0%}.\n'
        f'Duration: {anomaly.duration_seconds or "unknown"} seconds.\n'
        f'Biometrics at time of event:\n'
        f'  Heart rate: {sensor.heart_rate} bpm\n'
        f'  SpO2: {sensor.spo2}%\n'
        f'  Skin temperature: {sensor.skin_temperature}°C\n'
        f'  Step count: {sensor.step_count}\n'
        f'Location context: {location}.\n'
        'Generate a structured incident report.'
    )

prompt = build_user_prompt(sample_anomaly, sample_sensor, location_context)
print(prompt)

## 5. LLM Call & Response Parsing

In [ ]:
# Set your API key — use Colab secrets or environment variable
# from google.colab import userdata
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

client = OpenAI()  # reads OPENAI_API_KEY from env

response = client.chat.completions.create(
    model='gpt-4o-mini',
    temperature=0.3,  # TODO: fine-tune — lower for more deterministic outputs
    messages=[
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': prompt},
    ],
)

raw_output = response.choices[0].message.content
print('=== Raw LLM Output ===')
print(raw_output)

## 6. Parse into IncidentReport Schema

In [ ]:
# HACKATHON: simplify — use structured outputs / function-calling post-hackathon
def parse_report(text: str) -> dict:
    fields = {
        'severity': 'MODERATE',
        'summary': 'Anomaly detected. Manual review recommended.',
        'vitals_assessment': 'Sensor data collected.',
        'cv_assessment': f'CV detected {sample_anomaly.type} event.',
        'recommended_action': 'Check on individual.',
    }
    for line in text.splitlines():
        line = line.strip()
        for key in fields:
            prefix = f'{key}:'
            if line.lower().startswith(prefix.lower()):
                fields[key] = line[len(prefix):].strip()
    return fields

parsed = parse_report(raw_output)
report = IncidentReport(
    report_id=str(uuid.uuid4()),
    generated_at=int(time.time() * 1000),
    **parsed,
)

print('=== Structured Incident Report ===')
print(report.model_dump_json(indent=2))

## 7. Export Notes → FastAPI

| Notebook cell | Maps to FastAPI file |
|---|---|
| `SYSTEM_PROMPT` + `build_user_prompt()` | `cv_backend/routers/report.py` |
| `SensorSnapshot`, `AnomalyEvent`, `IncidentReport` | `cv_backend/schemas/` |
| `parse_report()` | `cv_backend/routers/report.py` → `_parse_llm_response()` |

**Next steps:**
- Switch to OpenAI structured outputs (JSON schema) for reliable parsing
- Add severity mapping based on biometric thresholds
- Experiment with GPT-4o for higher quality triage reasoning